In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from scipy.spatial.distance import cdist #for euclidean distance of consistency
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import uuid #to generate random id
import pickle #to save/load list of selected hens
import cv2

#test equal variance
from scipy.stats import levene

#clustering
from sklearn.cluster import AgglomerativeClustering 

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#interpolate curves for clustering among birds with not exact same days tracked
#from scipy.interpolate import interp1d

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score, r2_score,\
mean_squared_error, mean_absolute_error, explained_variance_score#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
from scipy.stats import pearsonr, spearmanr 
import scipy.stats as stats

#PCA
from sklearn import decomposition

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns
import dexplot as dxp #for barplot
from mpl_toolkits.mplot3d import Axes3D

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, ts_visual, time_series_henColumn_tsRow, correct_key, FB_daily
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [8]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_dataoutput = config.path_dataoutput
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_HAID_date = config.dico_HAID_date
dico_window = config.dico_window
path_extracted_HA = config.path_extracted_HA
path_extracted_HA_visual = config.path_extracted_HA_visual
path_extracted_data_SNA = config.path_extracted_data_SNA
path_extracted_data_visual_adap = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability')
path_extracted_data_visual_adap_clustering = os.path.join(path_extracted_data_visual_adap,'clustering')
V = 'chi2distance_ALL_15'
name_clustering = 'ALL_MLPS_99'
path_extracted_data_visual_adap_clustering_V = os.path.join(path_extracted_data_visual_adap_clustering, V+'_ALL_MLPS')
#create a director if not existing
if not os.path.exists(path_extracted_HA_visual):
    os.makedirs(path_extracted_HA_visual)
print(id_run)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


change the configuration file if not done yet!
correctlightschedule_


# Download variables

In [4]:
#network var
df_SNA = pd.read_csv(os.path.join(path_extracted_data,id_run+'_df_SNA.csv'), sep=';', parse_dates=['date'], dayfirst=True) 
df_SNA.rename({'date':'level'},axis=1,inplace=True)
print(df_SNA.shape)
display(df_SNA.head(3))
li_df = []
for v in ['closeness', 'betweeness', 'degree','WeightedDegree']:
    df_n = df_SNA.pivot(index=['HenID', 'level'], columns='distance_measure', values=v).reset_index()
    df_n.rename({x:v+'_'+x for x in df_n.columns if x.startswith('DTW_')}, axis=1, inplace=True)
    li_df.append(df_n)
#only display info on the last df    
print(df_n.shape)
display(df_n.head(3))
df_SNA_daily = pd.merge(li_df[0], li_df[1], on=['HenID','level'], how='outer')
df_SNA_daily = pd.merge(df_SNA_daily, li_df[2], on=['HenID','level'], how='outer')
print(df_SNA_daily.shape)
df_SNA_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(192192, 6)


,HenID,closeness,betweeness,degree,level,distance_measure
0,hen_131,0.569170,0.002268,0.368056,2020-09-30,DTW_30_all
1,hen_134,0.489796,0.000927,0.361111,2020-09-30,DTW_30_all
2,hen_138,0.578313,0.003665,0.388889,2020-09-30,DTW_30_all


(32032, 8)


distance_measure,HenID,level,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all
0,hen_1,2020-09-30,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500
1,hen_1,2020-10-01,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952
2,hen_1,2020-10-02,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401


(32032, 20)


distance_measure,HenID,level,closeness_DTW_30_1_Zone,closeness_DTW_30_2_Zone,closeness_DTW_30_3_Zone,closeness_DTW_30_4_Zone,closeness_DTW_30_5_Zone,closeness_DTW_30_all,betweeness_DTW_30_1_Zone,betweeness_DTW_30_2_Zone,betweeness_DTW_30_3_Zone,betweeness_DTW_30_4_Zone,betweeness_DTW_30_5_Zone,betweeness_DTW_30_all,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all
0,hen_1,2020-09-30,1.000000,0.320713,0.502045,0.812132,0.529412,0.562500,0.0,0.009265,0.001009,0.000124,0.006278,0.002710,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500
1,hen_1,2020-10-01,0.993197,0.357117,0.511829,0.769158,0.475432,0.577716,0.0,0.190033,0.000484,0.000074,0.015672,0.183609,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952
2,hen_1,2020-10-02,1.000000,0.415492,0.695425,0.791667,0.508651,0.542435,0.0,0.000154,0.000588,0.000069,0.001728,0.000955,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401


In [5]:
#daily variables (one row per (henID, date))
df_daily = pd.read_csv(os.path.join(path_extracted_data, 'daily_ALL_Variable_Tranformed.csv'), sep=';',
                     parse_dates=['level'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by date
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]
print(df_daily.shape)

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(42103, 218)
(42099, 218)
(42099, 218)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,clustering_ALL,weeks_in_laying_barn
0,hen_1,2020-09-30,0.0,18886.0,3488.0,515.0,5911.0,28800.0,"[0.0, 18886.0, 3488.0, 515.0, 5911.0]","[0.0, 0.6557638888888889, 0.12111111111111111,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,"[0.0, 9310.0, 5636.0, 0.0, 17454.0]","[0.0, 0.2873456790123457, 0.17395061728395061,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,"[0.0, 0.0, 551.0, 0.0, 31849.0]","[0.0, 0.0, 0.017006172839506173, 0.0, 0.982993...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [9]:
#daily clusterID
df_res = pd.read_csv(os.path.join(path_extracted_data_visual_adap_clustering_V,id_run+'df_clustering_results.csv'), sep=';',
                     dayfirst=True) 
df_res['DIB'] = df_res['MLPID'].map(lambda x: int(x.split('-')[1]))
df_res['date'] = df_res['DIB'].map(lambda x: dt.datetime(2020,9,29)+dt.timedelta(days=x))
df_res['HenID'] = df_res['MLPID'].map(lambda x: x.split('-')[0])
df_res = df_res.rename(columns={'date':'level'})
print(df_res.shape)
df_res.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(5146, 102)


,MLPID,ALL_MLPS_2,ALL_MLPS_3,ALL_MLPS_4,ALL_MLPS_5,ALL_MLPS_6,ALL_MLPS_7,ALL_MLPS_8,ALL_MLPS_9,ALL_MLPS_10,...,ALL_MLPS_93,ALL_MLPS_94,ALL_MLPS_95,ALL_MLPS_96,ALL_MLPS_97,ALL_MLPS_98,ALL_MLPS_99,DIB,date,HenID
0,hen_61-20,1,1,3,1,0,0,2,0,5,...,67,67,67,67,67,67,67,20,2020-10-19,hen_61
1,hen_65-35,0,0,1,0,5,5,5,5,2,...,9,9,9,4,4,1,0,35,2020-11-03,hen_65
2,hen_138-54,1,1,3,1,0,6,6,2,0,...,64,64,64,64,64,64,64,54,2020-11-22,hen_138


# Add SNA

In [28]:
#merge with dailySNA
#df_daily and df_SNA_daily do not have the same lenght, as sometimes we have in df_daily the nights variables, while never
#in df_SNA_daily
df = pd.merge(df_daily, df_SNA_daily, on=['HenID','level'], how='outer')
print(df_daily.shape, df_SNA_daily.shape, df.shape)
#check if we have some days with daily var in df_daily, but with no SNA
if df[(df['closeness_DTW_30_1_Zone'].isnull())&(~df['verification_daily_total_duration'].isnull())].shape[0]>0:
    print('you have not the same amount of SNA and DAILY observations (HenID, level)')
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(42099, 218) (32032, 20) (42300, 236)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,betweeness_DTW_30_3_Zone,betweeness_DTW_30_4_Zone,betweeness_DTW_30_5_Zone,betweeness_DTW_30_all,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all
0,hen_1,2020-09-30,0.0,18886.0,3488.0,515.0,5911.0,28800.0,"[0.0, 18886.0, 3488.0, 515.0, 5911.0]","[0.0, 0.6557638888888889, 0.12111111111111111,...",...,0.001009,0.000124,0.006278,0.002710,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,"[0.0, 9310.0, 5636.0, 0.0, 17454.0]","[0.0, 0.2873456790123457, 0.17395061728395061,...",...,0.000484,0.000074,0.015672,0.183609,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,"[0.0, 0.0, 551.0, 0.0, 31849.0]","[0.0, 0.0, 0.017006172839506173, 0.0, 0.982993...",...,0.000588,0.000069,0.001728,0.000955,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401


In [29]:
#remove columns with list or that was here simply to compute/verify other variables, as wew ill now not use these variables 
df = df.drop(['dur_values','dur_values_normalized','previous_dur_values_normalized', 'FirstTimestamp_1_Zone', 
              'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone', 'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone',
              'list_timestamps', 'list_timestamps_seondsOfTheDay','list_of_night20_2_temperature',
              'list_of_MovementCounter_day','list_of_durations','list_of_zones','t_DU_missingZone_mvtPerc', 
              'li_event_chaoticmvt_z_d'],axis=1)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
#merge with daily clusterID
#for now not same amount of level
li_name_clustering = ['ALL_MLPS_99','ALL_MLPS_10','ALL_MLPS_40']
print(len(df['level'].unique()), len(df_res['level'].unique()))
df = pd.merge(df, df_res[li_name_clustering+['HenID','level']], on=['HenID','level'], how='outer')
print(df_daily.shape, df_SNA_daily.shape, df.shape)
#check if we have some days with daily var in df_daily, but with no SNA
if df[(df['closeness_DTW_30_1_Zone'].isnull())&(~df['verification_daily_total_duration'].isnull())].shape[0]>0:
    print('you have not the same amount of SNA and DAILY observations (HenID, level)')
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


219 40
(42099, 218) (32032, 20) (42300, 223)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,Total_number_zone,Max_duration_zones,...,betweeness_DTW_30_all,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all,ALL_MLPS_99,ALL_MLPS_10,ALL_MLPS_40
0,hen_1,2020-09-30,0.0,18886.0,3488.0,515.0,5911.0,28800.0,4.0,2_Zone,...,0.002710,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500,NaN,NaN,NaN
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,3.0,5_Zone,...,0.183609,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952,NaN,NaN,NaN
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,2.0,5_Zone,...,0.000955,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401,NaN,NaN,NaN


In [31]:
#save
df.to_csv(os.path.join(path_extracted_data,'daily_ALL_Variable_Tranformed_SNA_clID.csv'), index=False, sep=';')

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
print(list(df.columns))

['HenID', 'level', 'duration_1_Zone', 'duration_2_Zone', 'duration_3_Zone', 'duration_4_Zone', 'duration_5_Zone', 'verification_daily_total_duration', 'Total_number_zone', 'Max_duration_zones', 'dico_zone_sortedduration', 'Total_number_transition', 'distribution_entropy', 'vertical_travel_distance', 'nbr_stays_3_Zone', 'nbr_appearances_3_Zone', 'empproba_3_Zone', 'nbr_stays_4_Zone', 'nbr_appearances_4_Zone', 'empproba_4_Zone', 'nbr_stays_5_Zone', 'nbr_appearances_5_Zone', 'empproba_5_Zone', 'nbr_stays_2_Zone', 'nbr_appearances_2_Zone', 'empproba_2_Zone', 'nbr_stays_1_Zone', 'nbr_appearances_1_Zone', 'empproba_1_Zone', 'Max_duration_zone_4', 'down_missingZone_mvtPerc', 'up_missingZone_mvtPerc', 'mean_duration', 'median_duration', 'max_duration', 'min_duration', 'variance_duration', 'percentile_5', 'percentile_15_duration', 'percentile_85_duration', 'percentile_95_duration', 'median_abs_deviation_duration', 'kurtosis_duration', 'skew_duration', 'stay_longer_60sec', 'stay_longer_60sec_3_Z

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
